In [2]:
from src import BM25Index
from utils import DocumentDB

db = DocumentDB()
bm25 = BM25Index()

print("loaded")

loaded


In [3]:
import pytrec_eval
from utils import read_qrel_from_file, evaluate_run, extract_topics_from_file
topics = extract_topics_from_file("qrels/topics_train.txt")
qrel = read_qrel_from_file("qrels/qrel_train.txt")

metrics: set = {'map', 'ndcg', 'P_5'}
evaluator = pytrec_eval.RelevanceEvaluator(qrel, metrics)

import numpy as np

def print_cum_stats(run):
    run_results = evaluator.evaluate(run)

    map_scores = [v["map"] for k,v in run_results.items()]
    p_scores  = [v["P_5"] for k,v in run_results.items()]
    ndcg_scores = [v['ndcg'] for k,v in run_results.items()]

    print("Aggregate results")
    print("Average MAP: ", np.mean(map_scores))
    print("Average P_5: ", np.mean(p_scores))
    print("Average NDCG: ", np.mean(ndcg_scores))


In [17]:
import spacy 

nlp = bm25.nlp

In [89]:
from tqdm.notebook import tqdm
from nltk.tokenize import sent_tokenize
from textblob import TextBlob
from collections import Counter

causitives = ["because", "because of", "due to", "lead to", "causes", "result of", "leading cause"]
causitive_re = re.compile(r"|".join(causitives))

ignore = ["irrelevant", "not relevant", "does not satisfy relevance"]

def extract_causal_terms(texts, num_terms: int = 100):
    terms = []
    for text in tqdm(texts):
        sents  = sent_tokenize(text)
        for sent in sents:
            snorm = sent.lower()
            if causitive_re.search(snorm):
                blob = TextBlob(snorm)
                verbs = [ tup[0] for tup in blob.tags 
                            if tup[1] in verb_tags and tup[0] not in STOP_WORDS ]
                terms.extend(list(blob.noun_phrases))
    terms = [tup[0] for tup in Counter(terms).most_common(num_terms)]
    return terms
    

In [97]:
from collections import Counter
from tqdm.notebook import tqdm

run = {}
for topic in tqdm(topics, total = len(topics)):
    # 1st search
    title = topic["title"].strip()
    bti, bts = bm25.query(title, num_results=100)
    texts = db.get_docs_from_ids(bti)
    terms = extract_causal_terms(texts, num_terms=50)

    # 2nd search
    new_query = title + " ".join(terms)
    bti2, bts2 = bm25.query(title, num_results=100)
    docnos = db.batch_idx_lookup(bti)

    final = dict(zip(docnos, bts2))
    run[topic["number"]] = final


In [98]:
print_cum_stats(run)

Aggregate results
Average MAP:  0.3672574509214016
Average P_5:  0.53
Average NDCG:  0.5836224277875107


In [88]:
db.batch_idx_lookup

['warning system',
 'international law',
 'fresh violence',
 'scuttle peace initiatives',
 'brutal assassination',
 'foreign minister lakshman kadirgamar',
 'strong signal',
 'such violent acts',
 'global isolation',
 'tamil tigers',
 'new generations',
 'sir richard attenborough',
 'recent mid-term elections']

In [74]:
import pke
from nltk.corpus import stopwords

extractor = pke.unsupervised.TopicRank()
extractor.load_document(input=topics[1]["narrative"].strip(), language='en')
extractor.candidate_selection()
extractor.candidate_weighting(threshold=.65)
extractor.get_n_best(n=15)

[('relevant documents', 0.10230590928214879),
 ('serial killing', 0.09846427343907829),
 ('blasts', 0.0956229885524526),
 ('mumbai terror attack', 0.09223428922046087),
 ('kasab', 0.0869510967834399),
 ('complicity', 0.08605254658766617),
 ('confession', 0.08484378351772881),
 ('news', 0.07431433667211303),
 ('trials', 0.07288941362966493),
 ('information', 0.0703752562262171),
 ('qaida group', 0.07016462055938852),
 ('family background', 0.06578148552964054)]

In [4]:
from flair.models import SequenceTagger
from flair.data import Sentence

model = SequenceTagger.load('tagger/causal-tagger/best-model.pt')

/home/dhairya.dalal/.conda/envs/nlp-env/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0
2021-06-13 14:32:06,854 loading file tagger/causal-tagger/best-model.pt


In [4]:
s = Sentence("As a result of smoking, people are more likely to get lung cancer")
model.predict(s)
for entity in s.get_spans('ner'):
    print(entity)


Span [5]: "smoking"   [− Labels: CAUSE (0.9431)]
Span [13,14]: "lung cancer"   [− Labels: EFFECT (0.7495)]


In [7]:
topics[0]["title"]

'Telecom minister A. Raja resignation'

In [10]:
topics[0]["narrative"]

'Documents retrieved should contain information about the accusations against A. Raja in the 2G Spectrum scam. Documents that talk about his criticisms for allocating 4.4MHz of spectrum  a scarce resource  at the 2001 price of Rs 1,651 crore through a first come, first served policy rather than auctions; by compromising on the market value of spectrum, the government is estimated to have lost revenues worth around Rs 50,000 crore; new licensees are signing equity deals with foreign players in which the value of spectrum has risen sevenfold, would be considered as relevant.\n'

In [8]:
qrel[topics[0]["number"]]

{'1101228_nation_story_13359273.utf8': 1,
 '1101224_nation_story_13342958.utf8': 1,
 '1101222_nation_story_13332987.utf8': 1,
 '1101218_nation_story_13316550.utf8': 1,
 '1101211_nation_story_13287065.utf8': 1,
 '1101210_frontpage_story_13282615.utf8': 1,
 '1101209_nation_story_13277777.utf8': 1,
 '1101203_nation_story_13253099.utf8': 1,
 '1101202_nation_story_13248318.utf8': 1,
 '1101130_opinion_story_13228753.utf8': 1,
 '1101126_nation_story_13224009.utf8': 1,
 '1101120_nation_story_13199747.utf8': 1,
 '1101119_frontpage_story_13195587.utf8': 1,
 '1101117_business_story_13186608.utf8': 1,
 '1101116_business_story_13182132.utf8': 1,
 '1101116_nation_story_13182738.utf8': 1,
 '1101113_nation_story_13171965.utf8': 1,
 '1101112_business_story_13167887.utf8': 1,
 '1101111_business_story_13163812.utf8': 1,
 '1101030_business_story_13118628.utf8': 1,
 '1100914_business_story_12934133.utf8': 1,
 '1100525_business_story_12484627.utf8': 1,
 '1100523_frontpage_story_12478906.utf8': 1,
 '1100512_

In [18]:
text = db.lookup_docno("1101119_frontpage_story_13195587.utf8")

In [19]:
text = Sentence(text)

In [20]:
model.predict(text)
for entity in text.get_spans('ner'):
    print(entity)


Span [77,78,79,80]: "loss to the exchequer"   [− Labels: EFFECT (0.8522)]
Span [86]: "decision-making"   [− Labels: CAUSE (0.9859)]
Span [870]: "loss"   [− Labels: EFFECT (0.9411)]
Span [1006]: "records"   [− Labels: CAUSE (0.8411)]


In [21]:
db.lookup_docno("1101119_frontpage_story_13195587.utf8")

'The Telegraph - Calcutta (Kolkata) | Frontpage | Sprinkle some zeroes to spice up a scam 1 Sprinkle some zeroes to spice up a scam OUR BUREAU Nov. 18: The spectrum controversy has embedded one figure in public consciousness: Rs 1.76 lakh crore, something that prompted Jayalalithaa to say the common man will be hard-pressed to find out how many zeroes do you add to this number. Thats the presumed loss to the exchequer as a result of the decision-making by A. Raja, who has resigned as telecom minister, while issuing telecom licences and 2G spectrum in January 2008. But what is lost in the cacophony is that this figure is based on some dodgy presumptions and raises questions about the number-crunching methods adopted by the audit sleuths at the Comptroller and Auditor General (CAG). The CAG did put out four figures in its report while calculating the notional loss to the exchequer. It had worked in the caveats as well but the largest number appears to have hogged the limelight. The big b

In [26]:
ids, _ = bm25.query(topics[0]["title"], num_results=10)
docnos = db.batch_idx_lookup(ids)

In [28]:
texts = db.batch_docno_lookup(docnos)

In [29]:
for text in texts:
    print(text)
    print("extracted relations")
    
    s = Sentence(text)
    model.predict(s)
    for entity in
        print(entity)


    print("------------------------------------")



esperation to wean the Congress away from the DMK fold. Now, however, it looks like a tactical masterstroke. The average Tamil would perceive that Raja resigned only after our leader made that dramatic announcement on TV. Even if the Congress does not come to our side, weve scored an important psychological victory over the DMK, an AIADMK MP said.
extracted relations
Span [328,329,330]: "offer of support"   [− Labels: CAUSE (0.7215)]
Span [334,335,336,337,338,339]: "Congress into some kind of action"   [− Labels: EFFECT (0.8452)]
------------------------------------
The Telegraph - Calcutta (Kolkata) | Business | Raja to clear the air on spectrum Tuesday , November 3 , 2009 Raja to clear the air on spectrum OUR CORRESPONDENT New Delhi, Nov. 2: Telecom minister A. Raja will meet the heads of telecom companies and Internet service providers from tomorrow to discuss issues such as spectrum allocation and the CBI probe into irregularities in the grant of licences. Sources said the meetings

In [ ]:
# Step 1 : query system with title
# Step 2: Run casual extractor over results a
# Step 3: Keep results where extracted effects are similiar to the query event 


In [6]:
from collections import Counter
from tqdm.notebook import tqdm

run = {}
for topic in tqdm(topics, total = len(topics)):

    title = topic["title"].strip()
    number = topic["number"]

    # Step 1 : query system with title
    initial_results, _  = bm25.query(title, num_results=10)
    initial_docnos = db.batch_idx_lookup(initial_results)
    initial_texts = db.batch_docno_lookup(initial_docnos)
    
    # Step 2: Run casual extractor over results a
    filted_docs = {}

    for idx, doc in enumerate(initial_texts):
        sent = Sentence(doc)
        model.predict(sent)

        for relation in sent.get_spans('ner'):
           
            if relation.tag == "EFFECT" and relation.text.lower() in title.lower():
                doc_id = initial_docnos[idx]
                score = _[idx]
                filted_docs[doc_id] = score

    # Step 3: Keep results where extracted effects are similiar to the query event 
    run[number] = filted_docs

print_cum_stats(run)


Aggregate results
Average MAP:  0.008823529411764706
Average P_5:  0.08
Average NDCG:  0.02875344092193561


In [11]:
for doc_id, score in run["16"].items():
    print(doc_id in qrel["16"], score, doc_id)

True 24.68444401109895 1091031_business_story_11682361.utf8
True 23.32012861876966 1100525_business_story_12484627.utf8
False 23.02927069348644 1101115_nation_story_13178225.utf8
False 22.77542627320952 1101113_frontpage_story_13171996.utf8
True 22.295120974420474 1101116_nation_story_13182738.utf8
False 21.917628095873006 1091103_business_story_11692612.utf8
False 21.713110664411893 1091027_nation_story_11662903.utf8
False 21.666311084822727 1101204_nation_story_13257636.utf8
True 21.647100653299695 1091023_business_story_11648333.utf8
True 20.68277116666247 1101113_nation_story_13171965.utf8


In [12]:
db.lookup_docno("1101115_nation_story_13178225.utf8")

'The Telegraph - Calcutta (Kolkata) | Nation | Raja departure whets Cong appetite 2 Raja departure whets Cong appetite RADHIKA RAMASESHAN AND G.C. SHEKHAR Manmohan Singh (top), M Karunanidhi Nov. 14: If Jayalalithaas offer helped the Centre to persuade M. Karunanidhi to display good sense and ease out A. Raja, the Congress is banking on the same factor to make a run for the telecom portfolio. The formal procedure to accept Rajas resignation will kick in tomorrow morning after the Prime Minister forwards the letter to the President. The Prime Minister is likely to hold the telecom portfolio until after Parliaments winter session is over. He is expected to carry out a larger reshuffle before the AICC plenary to fill the vacancies left by the departure of Shashi Tharoor, Prithviraj Chavan and now Raja. Congress sources said the party might leverage its newfangled clout and try to get the telecom portfolio, given the potential of the berth to stir controversies. The portfolio has remained 

In [14]:
d = 'The Telegraph - Calcutta (Kolkata) | Nation | Raja departure whets Cong appetite 2 Raja departure whets Cong appetite RADHIKA RAMASESHAN AND G.C. SHEKHAR Manmohan Singh (top), M Karunanidhi Nov. 14: If Jayalalithaas offer helped the Centre to persuade M. Karunanidhi to display good sense and ease out A. Raja, the Congress is banking on the same factor to make a run for the telecom portfolio. The formal procedure to accept Rajas resignation will kick in tomorrow morning after the Prime Minister forwards the letter to the President. The Prime Minister is likely to hold the telecom portfolio until after Parliaments winter session is over. He is expected to carry out a larger reshuffle before the AICC plenary to fill the vacancies left by the departure of Shashi Tharoor, Prithviraj Chavan and now Raja. Congress sources said the party might leverage its newfangled clout and try to get the telecom portfolio, given the potential of the berth to stir controversies. The portfolio has remained with the DMK under the first and second UPAs Dayanidhi Maran holding charge initially and then Raja straddling both the editions. But DMK sources said the party would insist on retaining the portfolio. Among the DMK aspirants for the coveted berth is T.R. Baalu, who lost out in the ministerial race two years ago, and he has been pushing his case with Karunanidhi. But deputy chief minister M.K Stalin is reportedly backing minister of state for finance S.S. Palanimanickam and wants him to be elevated to cabinet rank. Stalins contention is that Palanimanickam has been minister of state for more than seven years now and that experience would come in handy in a crucial portfolio like telecom. But the chances of A.K. Vijayan, a three-time MP from Nagapattinam, looks the brightest as he is a Dalit like Raja. The Centre and the Congress appear to have reckoned that even if the Supreme Court censured Raja again tomorrow, the negative impact would be offset by the resignation. The court will hear two petitions filed against him over the allocation of 2G spectrum. If restoring normality in Parliament that did not function on the four working days last week was the immediate objective of the UPA, the long-term aim, said sources, was to retrieve the sense of iqbal (grace) the government seems to have lost in some measure over the past few days as Raja dug in his heels. In public perception, some people are held guilty and Raja is one of them. If we sit tight and do nothing, we will be perceived as co-conspirators out to support those perceived as guilty. Hardly the sort of thing the Prime Minister would want to live with, a cabinet minister had said a few hours before Raja put in his papers. The Congress tried hard to keep the moral ground by setting itself apart from errant allies and showing up the ouster of Suresh Kalmadi and Ashok Chavan as examples. However, a source acknowledged that the argument of being compelled to stick to the coalition dharma did not wash once the BJP dragged Singh into its campaign. It is a tricky situation because people, especially the educated middle-class, who hold the Prime Minister in high regard, wonder why he is so helpless and cannot enforce his writ over corrupt ministers, a Congress Rajya Sabha MP from Tamil Nadu said in the evening. These circumstances were discussed when the Congress leadership met this morning in Parliament after paying tributes to Jawaharlal Nehru on his birth anniversary. Sources said the leaders apart from Singh, Sonia, Pranab Mukherjee and Ahmed Patel were also concerned about the ramifications on the DMKs first family politics. That Raja was backed by Rajya Sabha MP and Karunanidhis daughter Kanimozhi and her mother is no secret. But when Kanimozhis name was considered as a replacement, her half-brothers M.K. Stalin and M.K. Alagiri saw red, afraid that her rise as an independent power centre might be legitimised. After tasting success on Raja, the BJP said it would continue the campaign against corruption but without disrupting Parliament. Parliament (disruption) is a small thing. It is public opinion and our persuasion that eventually triumphed and forced the Prime Minister to coax out Rajas resignation. Henceforth, the UPA government will be on permanent notice. There will be no let-up in our pressure, said BJP spokesperson and Rajya Sabha MP Prakash Javadekar. BJP sources said had they not targeted the Prime Minister directly and asked tough questions on why he was quiet for so long, nothing would have happened. However, the AIADMK said it would continue demanding a joint parliamentary committee probe into the 2G scam. A worry for the Congress stems from the possibility that the Opposition might smell blood and raise the scope of its demands to include others such as power minister S.K. Shinde. Activist Medha Patkar and a former police officer had complained that Shinde had allegedly manipulated documents to push his relatives case in the Adarsh housing allotment. To counter the offensive, the Congress is readying dossiers in a land allocation case in the BJP-ruled Karnataka in which chief minister B.S. Yeddyurappa is alleged to have favoured his sons. Asked if Yeddyurappa would be asked to step down, Javadekar said: Why should he? There is no comparison.'
s = Sentence(d)

model.predict(s)
for entity in s.get_spans('ner'):
    print(entity)



Span [176]: "controversies"   [− Labels: EFFECT (0.5998)]
Span [354,355]: "negative impact"   [− Labels: EFFECT (0.7996)]
Span [361]: "resignation"   [− Labels: CAUSE (0.8983)]
Span [758]: "persuasion"   [− Labels: CAUSE (0.7469)]
Span [914]: "offensive"   [− Labels: EFFECT (0.7078)]


In [7]:
from collections import Counter
from tqdm.notebook import tqdm

run = {}
for topic in tqdm(topics, total = len(topics)):

    title = topic["title"].strip()
    number = topic["number"]

    # Step 1 : query system with title
    initial_results, _  = bm25.query(title, num_results=10)
    initial_docnos = db.batch_idx_lookup(initial_results)
    initial_texts = db.batch_docno_lookup(initial_docnos)
    
    results = dict(zip(initial_docnos, _))

    # Step 2: Run casual extractor over results a
    # filted_docs = {}

    # for idx, doc in enumerate(initial_texts):
    #     sent = Sentence(doc)
    #     model.predict(sent)

    #     for relation in sent.get_spans('ner'):
           
    #         if relation.tag == "EFFECT" and relation.text.lower() in title.lower():
    #             doc_id = initial_docnos[idx]
    #             score = _[idx]
    #             filted_docs[doc_id] = score

    # Step 3: Keep results where extracted effects are similiar to the query event 
    run[number] = results

print_cum_stats(run)


Aggregate results
Average MAP:  0.3482026053581781
Average P_5:  0.6400000000000001
Average NDCG:  0.4856021664613019
